In [2]:
import datetime as dt
import pandas as pd

In [3]:
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas105semanas.parquet")

In [4]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)


df["client_name"] = df["client_name_inv"].str.split("] ", expand=True)[1].str.strip()



#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])

df = df.drop(columns=["origin"])

df["product_ref"] = df["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df["product_nom"] = df["product_name"].str.extract(r"\]\s*([A-Z]\w+)")


df["date_invoice"] = pd.to_datetime(df["date_invoice"])
df["mes"] = df["date_invoice"].dt.month
df["año"] = df["date_invoice"].dt.year
df["dia"] = df["date_invoice"].dt.day

df = df[~(df["product_ref"].isna())]

pd.set_option('display.max_columns', None)

df['marca'] = df['product_ref'].apply(
    lambda x: 'BALDWIN' if str(x).endswith('125') else
              'DONSSON' if str(x).endswith('025') else
              'AUT*PARTS' if str(x).endswith('189') else
              'RACOR BRASIL' if str(x).endswith('137') else
              'RACOR USA' if str(x).endswith('138') else
              'OTRA'
)

In [13]:
df[df["client_name"].str.contains("VILMAR")]

,product_id,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv,Sucursal_tipo,client_name,origen,Bodega,product_ref,product_nom,mes,año,dia,marca
3861,"[17298, '[DAB02977025] DA2977 FILTRO AIRE KIA ...",64200.0,1627808,2.0,[DAB02977025] DA2977 FILTRO AIRE KIA PREGIO GR...,FCOT26514,2025-10-04 17:04:21,FCOT26514,441492,PRINCIPAL COTA,HAROLD JIMENEZ,EXTERNOS,3019,[CO9006690798] VILMAR LUBRICANTES S.A.S.,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,BD1,PRINCIPAL COTA,DAB02977025,DA2977,10,2025,4,DONSSON
3862,"[49909, '[DLS00295189] GS295 FILTRO ACEITE DUA...",125400.0,1627814,6.0,[DLS00295189] GS295 FILTRO ACEITE DUAL - CHEVR...,FCOT26514,2025-10-04 17:04:21,FCOT26514,441492,PRINCIPAL COTA,HAROLD JIMENEZ,EXTERNOS,3019,[CO9006690798] VILMAR LUBRICANTES S.A.S.,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,BD1,PRINCIPAL COTA,DLS00295189,GS295,10,2025,4,AUT*PARTS
3863,"[17670, '[DAB04690025] DA4690 FILTRO AIRE 2o J...",76800.0,1627809,2.0,"[DAB04690025] DA4690 FILTRO AIRE 2o J.DEERE, F...",FCOT26514,2025-10-04 17:04:21,FCOT26514,441492,PRINCIPAL COTA,HAROLD JIMENEZ,EXTERNOS,3019,[CO9006690798] VILMAR LUBRICANTES S.A.S.,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,BD1,PRINCIPAL COTA,DAB04690025,DA4690,10,2025,4,DONSSON
3864,"[16994, '[DAR02499025] DA2499 FILTRO AIRE DONS...",379200.0,1627799,12.0,[DAR02499025] DA2499 FILTRO AIRE DONSSON - Che...,FCOT26514,2025-10-04 17:04:21,FCOT26514,441492,PRINCIPAL COTA,HAROLD JIMENEZ,EXTERNOS,3019,[CO9006690798] VILMAR LUBRICANTES S.A.S.,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,BD1,PRINCIPAL COTA,DAR02499025,DA2499,10,2025,4,DONSSON
3865,"[47444, '[DCS00235189] GS235 FILTRO COMBUSTIBL...",236400.0,1627815,12.0,[DCS00235189] GS235 FILTRO COMBUSTIBLE - TOYOT...,FCOT26514,2025-10-04 17:04:21,FCOT26514,441492,PRINCIPAL COTA,HAROLD JIMENEZ,EXTERNOS,3019,[CO9006690798] VILMAR LUBRICANTES S.A.S.,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,BD1,PRINCIPAL COTA,DCS00235189,GS235,10,2025,4,AUT*PARTS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371660,"[19115, '[BCS00536125] GS536A10 FILTRO COMBUST...",329400.0,1188005,3.0,[BCS00536125] GS536A10 FILTRO COMBUSTIBLE SEPA...,FCOT16432,2023-10-13 15:17:21,FCOT16432,323626,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,3019,[CO9006690798] VILMAR LUBRICANTES S.A.S.,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,BD1,PRINCIPAL COTA,BCS00536125,GS536A10,10,2023,13,BALDWIN
371661,"[18568, '[BLS00001125] GS001 FILTRO ACEITE FOR...",69900.0,1188002,3.0,"[BLS00001125] GS001 FILTRO ACEITE FORD,TOYOTA,...",FCOT16432,2023-10-13 15:17:21,FCOT16432,323626,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,3019,[CO9006690798] VILMAR LUBRICANTES S.A.S.,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,BD1,PRINCIPAL COTA,BLS00001125,GS001,10,2023,13,BALDWIN
371662,"[19045, '[BCS00191125] GS191 FILTRO COMBUSTIBL...",192400.0,1188004,2.0,[BCS00191125] GS191 FILTRO COMBUSTIBLE INTERNA...,FCOT16432,2023-10-13 15:17:21,FCOT16432,323626,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,3019,[CO9006690798] VILMAR LUBRICANTES S.A.S.,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,BD1,PRINCIPAL COTA,BCS00191125,GS191,10,2023,13,BALDWIN
371663,"[18734, '[BHS00167125] GS167 FILTRO ACEITE CAT...",225600.0,1188003,6.0,[BHS00167125] GS167 FILTRO ACEITE CATERPILLAR ...,FCOT16432,2023-10-13 15:17:21,FCOT16432,323626,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,3019,[CO9006690798] VILMAR LUBRICANTES S.A.S.,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,BD1,PRINCIPAL COTA,BHS00167125,GS167,10,2023,13,BALDWIN


In [6]:
from datetime import datetime

# Fecha y hora actual en formato YYYYMMDD_HHMMSS
timestamp = datetime.now().strftime("%Y%m%d")

# Nombre dinámico
#filename = f"/home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_{timestamp}.xlsx"

# Guardar el Excel
#df.to_excel(filename, index=False)

#print(f"Archivo guardado: {filename}")

#Tiempo promedio 3 minutos

## VISUM

In [14]:
clientes = {"VISUM LTDA",
"VILMAR LUBRICANTES S.A.S.",
"ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.", 
"SODATRANS LTDA",
"ESTACION Y LUBRICANTES CASABLANCA S.A.S"}

fecha_inicial = "09/1/2024"
fecha_final = "09/30/2025"

df_h = df[(df["client_name"].isin(clientes)) & (df["date_invoice"] >= fecha_inicial) & (df["date_invoice"] <= fecha_final) & (df["salesperson_name"] == "YEIS OSWALDO MUÑOZ GUERRERO")  ]

df_h = df_h.drop(columns=["product_id","id_x","id_y","partner_id_num","client_name_inv","invoice_name","origen","number"])

In [15]:
df_h["date_invoice"].max()

Timestamp('2025-09-19 14:59:01')

In [16]:
df_h.sample(10)

,price_subtotal,quantity,product_name,date_invoice,store_name,salesperson_name,sales_team_name,Sucursal_tipo,client_name,Bodega,product_ref,product_nom,mes,año,dia,marca
144632,251200.0,1.0,[BAP02020125] DA2020 FILTRO AIRE- CAT 416D (12...,2025-01-30 15:29:12,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BAP02020125,DA2020,1,2025,30,BALDWIN
95527,147600.0,4.0,[BLS00111125] GS111 FILTRO ACEITE - CUMMINS 6B...,2025-04-25 21:05:27,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BLS00111125,GS111,4,2025,25,BALDWIN
164088,61400.0,1.0,[BLS00297125] GS297 FILTRO ACEITE CENTRIFUGO M...,2024-12-18 13:42:37,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BLS00297125,GS297,12,2024,18,BALDWIN
164118,83000.0,1.0,[BLE00229125] G229 FILTRO ACEITE BY PASS HYUND...,2024-12-18 13:42:37,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BLE00229125,G229,12,2024,18,BALDWIN
193500,173400.0,6.0,"[BCS00035125] GS035 FILTRO COMBUSTIBLE VOLVO,D...",2024-10-24 16:25:06,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,ESTACION Y LUBRICANTES CASABLANCA S.A.S,PRINCIPAL COTA,BCS00035125,GS035,10,2024,24,BALDWIN
169650,115800.0,3.0,[DAB08161025] DA8161 FILTRO AIRE JAC 1035 MOD....,2024-12-09 20:42:34,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,DAB08161025,DA8161,12,2024,9,DONSSON
30934,224400.0,2.0,[DAB02982025] DA2982 FILTRO AIRE DONSSON - IH...,2025-08-21 13:56:51,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.,PRINCIPAL COTA,DAB02982025,DA2982,8,2025,21,DONSSON
193612,159600.0,3.0,[BCS00180125] GS180 FILTRO COMBUSTIBLE SEPARAD...,2024-10-24 16:02:08,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BCS00180125,GS180,10,2024,24,BALDWIN
187451,129500.0,5.0,[BCS10014125] GS014A FILTRO COMBUST. CUMMINS. ...,2024-11-06 14:12:53,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BCS10014125,GS014A,11,2024,6,BALDWIN
193607,162600.0,6.0,[DAB04783025] DA4783 FILTRO AIRE SEGURIDAD- C...,2024-10-24 16:02:08,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAB04783025,DA4783,10,2024,24,DONSSON


In [18]:
df_h[df_h["client_name"]=="VILMAR LUBRICANTES S.A.S."]

,price_subtotal,quantity,product_name,date_invoice,store_name,salesperson_name,sales_team_name,Sucursal_tipo,client_name,Bodega,product_ref,product_nom,mes,año,dia,marca
30985,724800.0,24.0,[DAB02788025] DA2788 FILTRO AIRE DONSSON - Ch...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAB02788025,DA2788,8,2025,21,DONSSON
30986,378000.0,6.0,[BLS00311125] GS311 FILTRO ACEITE BALDWIN - CU...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BLS00311125,GS311,8,2025,21,BALDWIN
30987,61600.0,2.0,[BCS10014125] GS014A FILTRO COMBUST. CUMMINS. ...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BCS10014125,GS014A,8,2025,21,BALDWIN
30988,83800.0,2.0,[BWS00078125] GS078A4 FILTRO REFRIGERACION CUM...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BWS00078125,GS078A4,8,2025,21,BALDWIN
30989,174800.0,2.0,[BLS00225125] GS225 FILTRO ACEITE TOYOTA MONTA...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BLS00225125,GS225,8,2025,21,BALDWIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209914,266400.0,6.0,[DAE04304025] DA4304 FILTRO AIRE 2_ CHEVROLET ...,2024-09-23 17:11:52,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAE04304025,DA4304,9,2024,23,DONSSON
209915,175200.0,6.0,[DAB04968025] DA4968 FILTRO AIRE 2_ HINO FC (0...,2024-09-23 17:11:52,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAB04968025,DA4968,9,2024,23,DONSSON
209916,186300.0,3.0,"[DAR08049025] DA8049 FILTRO AIRE HYUNDAI HD78,...",2024-09-23 17:11:52,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAR08049025,DA8049,9,2024,23,DONSSON
209917,136800.0,6.0,[DLS00295189] GS295 FILTRO ACEITE DUAL - CHEVR...,2024-09-23 17:11:52,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DLS00295189,GS295,9,2024,23,AUT*PARTS


In [19]:
df_h.to_excel("harold_vendedor.xlsx")